In [2]:
# 0314 v2 - 가까운 두 개 마커로 위치 파악
# 마커 개수 늘림 시야에 3개까지 잘 잡히는데 가까운 2개를 고르려다 보니까 어떤 마커 2개를 고르는지에 따라서 좌표가 다르게 나타남
#v3: 여러개 중에 2개씩 이용해서 위치 뽑고 평균 내기
#v4: 칼만 필터  - v3보다 개선됨 (튀는 값이 줄어들었지만 여전히 많긴 함)

# 0317 v1: z축 거리 고정해서 입력하기 + 칼만필터까지 적용
# 정확도 높음 z값 -1.8m로 했는데 로봇에 부착하고 다시 거리 재보면 될듯
# v2: 마커 17번 빠졌던 거 추가
# v3: 마커 z축 회전 정렬 
#v4: 회전 정렬 결과 나타나게 opencv 3d 축 그리기 반영

In [1]:
import cv2
import cv2.aruco as aruco
import numpy as np
from scipy.spatial.transform import Rotation as R

# ===========================
# 📌 1. 설정 및 초기값
# ===========================
cameraMatrix = np.load('C:/cal_cam/calibration_matrix.npy')
distCoeffs = np.load('C:/cal_cam/distortion_coefficients.npy')
marker_length = 0.16  # 마커 실제 길이 (m)
FIXED_Z = -1.8  # 고정된 Z 값 (m)

# 마커 월드 좌표
marker_world_pos = {
    0: np.array([0.0, 0.0, 0.0]), 1: np.array([1.5, 0.0, 0.0]),
    2: np.array([3.0, 0.0, 0.0]), 3: np.array([4.5, 0.0, 0.0]),
    4: np.array([0.0, 1.5, 0.0]), 5: np.array([1.5, 1.5, 0.0]),
    6: np.array([3.0, 1.5, 0.0]), 7: np.array([4.5, 1.5, 0.0]),
    8: np.array([0.0, 3.0, 0.0]), 9: np.array([1.5, 3.0, 0.0]),
    10: np.array([3.0, 3.0, 0.0]), 11: np.array([4.5, 3.0, 0.0]),
    12: np.array([0.7, 0.7, 0.0]), 13: np.array([2.3, 0.7, 0.0]),
    14: np.array([3.7, 0.7, 0.0]), 15: np.array([0.7, 2.3, 0.0]),
    16: np.array([2.3, 2.3, 0.0]), 17: np.array([3.7, 2.3, 0.0])
}

# 마커별 z축 회전 보정 (각도)
marker_orientation = {
    0: 0, 1: 0, 2: -180, 3: 0,
    4: -180, 5: -180, 6: 0, 7: -180,
    8: 0, 9: 0, 10: -180, 11: 0,
    12: -180, 13: -90, 14: -270,
    15: 0, 16: -90, 17: -90
}

# 칼만 필터 초기화 함수
def init_kalman_filter():
    kf = cv2.KalmanFilter(6, 3)
    kf.measurementMatrix = np.eye(3, 6, dtype=np.float32)
    kf.transitionMatrix = np.array([
        [1, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 1],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
    ], dtype=np.float32)
    kf.processNoiseCov = np.eye(6, dtype=np.float32) * 1e-4
    kf.measurementNoiseCov = np.eye(3, dtype=np.float32) * 1e-2
    return kf

kalman_filters = {}

# z축 회전 보정 행렬 생성 함수
def get_z_rotation_matrix(degrees):
    return R.from_euler('z', degrees, degrees=True).as_matrix()

# 카메라 위치 계산 함수
def compute_camera_pos_fixed_z_with_rotation(m_id, rvec, tvec, marker_world, fixed_z=FIXED_Z):
    R_marker, _ = cv2.Rodrigues(rvec)
    t = tvec.reshape(3, 1)
    t[2] = fixed_z  # z 고정
    cam_in_marker = -R_marker.T @ t

    # z축 보정 회전 적용
    correction_R = get_z_rotation_matrix(marker_orientation.get(m_id, 0))  # 없으면 0도
    cam_in_world = correction_R @ cam_in_marker + marker_world.reshape(3, 1)
    return cam_in_world

# ===========================
# 📌 2. 카메라 열기 및 메인 루프
# ===========================
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_EXPOSURE, -4)

aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
detector_params = aruco.DetectorParameters()

if not cap.isOpened():
    print("카메라 열기 실패!")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패!")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = aruco.detectMarkers(gray, aruco_dict, parameters=detector_params)

    if ids is not None:
        aruco.drawDetectedMarkers(frame, corners, ids)
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, marker_length, cameraMatrix, distCoeffs)

        id_to_pose = {id_[0]: (rvecs[i], tvecs[i]) for i, id_ in enumerate(ids)}
        found_ids = ids.flatten()

        cam_positions = []
        weights = []

        for m_id in found_ids:
            if m_id in marker_world_pos:
                rvec, tvec = id_to_pose[m_id]
                world_pos = marker_world_pos[m_id]

                # ✅ 원본 회전 -> 행렬
                R_marker, _ = cv2.Rodrigues(rvec)
                # ✅ 보정 회전
                correction_R = get_z_rotation_matrix(marker_orientation.get(m_id, 0))
                # ✅ 보정된 회전
                R_corrected = correction_R @ R_marker
                rvec_corrected, _ = cv2.Rodrigues(R_corrected)  # 다시 벡터로

                # ✅ 보정된 축 그리기
                cv2.drawFrameAxes(frame, cameraMatrix, distCoeffs, rvec_corrected, tvec, 0.1)

                # 보정 포함 카메라 위치 계산
                cam_pos = compute_camera_pos_fixed_z_with_rotation(m_id, rvec, tvec, world_pos).flatten()

                # 칼만 필터
                if m_id not in kalman_filters:
                    kalman_filters[m_id] = init_kalman_filter()
                kf = kalman_filters[m_id]
                kf.correct(cam_pos.astype(np.float32))  # 보정
                estimate = kf.predict()[:3]  # 예측

                cam_positions.append(estimate)
                distance = np.linalg.norm(tvec)
                weights.append(1.0 / (distance + 1e-6))

                # 마커 ID 표시
                c = corners[found_ids.tolist().index(m_id)][0][0]
                cv2.putText(frame, f"ID {m_id}", (int(c[0]), int(c[1] - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # 가중 평균으로 카메라 위치
        if cam_positions:
            weights = np.array(weights)
            weights /= np.sum(weights)
            cam_positions = np.array(cam_positions)
            camera_world_pos = np.average(cam_positions, axis=0, weights=weights)
            cx, cy, cz = camera_world_pos.ravel()
            print(f"[INFO] Camera world position (filtered): ({cx:.2f}, {cy:.2f}, {cz:.2f})")
            cv2.putText(frame, f"Camera Pos: X={cx:.2f}m, Y={cy:.2f}m, Z={cz:.2f}m", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


    # 화면 표시
    cv2.imshow("3D Axis Corrected ArUco", frame)

    # 'ESC' 키 누르면 종료
    if cv2.waitKey(1) & 0xFF == 27:
        break

# ===========================
# 📌 3. 종료 처리
# ===========================
cap.release()
cv2.destroyAllWindows()


[INFO] Camera world position (filtered): (0.00, 0.00, 0.00)
[INFO] Camera world position (filtered): (0.04, 0.01, -0.02)
[INFO] Camera world position (filtered): (0.21, -0.00, -0.08)
[INFO] Camera world position (filtered): (0.68, -0.03, -0.27)
[INFO] Camera world position (filtered): (1.50, -0.02, -0.62)
[INFO] Camera world position (filtered): (2.58, 0.00, -1.09)
[INFO] Camera world position (filtered): (3.66, -0.08, -1.54)
[INFO] Camera world position (filtered): (4.48, -0.16, -1.89)
[INFO] Camera world position (filtered): (5.02, -0.26, -2.11)
[INFO] Camera world position (filtered): (5.29, -0.33, -2.22)
[INFO] Camera world position (filtered): (5.40, -0.39, -2.25)
[INFO] Camera world position (filtered): (5.38, -0.42, -2.23)
[INFO] Camera world position (filtered): (5.49, -0.35, -2.20)
[INFO] Camera world position (filtered): (5.51, -0.29, -2.14)
[INFO] Camera world position (filtered): (5.31, -0.11, -2.09)
[INFO] Camera world position (filtered): (5.12, 0.02, -2.03)
[INFO] Camera